# Lab 10.4.2: Image Folder 2

Edited By Steve Ive

Here, with the custom dataset that we defined, we will create the model to classify our label(here, cat and dog) with our custom CNN.

Reference from

https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-10_4_2_ImageFolder_2.ipynb

## Imports

In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [75]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

## Define torchvision.transform property(To.Tensor()) to train

In [76]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root = 'catsanddogs/train_data/', transform=trans)

## Set Hyperparameter

In [77]:
batch_size = 8
training_epochs = 100
learning_rate = 0.01

## Define Dataloader

In [78]:
data_loader = torch.utils.data.DataLoader(dataset = train_data, shuffle = True, batch_size = batch_size, drop_last = False, num_workers = 2)

## Define Model
## Only Adding 1 more Fully connected Layer makes accuaracy 15% up!!!!!!!

50% -> 65%

In [79]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16 * 13 * 29, 120),
            nn.ReLU(),
            nn.Linear(120, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [80]:
#testing
net = CNN().to(device)
test_input = (torch.Tensor(3, 3, 64, 128)).to(device)
test_out = net(test_input)

In [81]:
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

## Train

In [82]:
total_batch = len(data_loader)

for epoch in range(training_epochs):

    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        #prediction
        pred = net(X)

        #cost
        cost = F.cross_entropy(pred, Y)

        #Reduce the cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost
    
    avg_cost = avg_cost / total_batch

    print('Epoch {} / {}, Cost: {:.6f}'.format(epoch+1, training_epochs, avg_cost))

Epoch 1 / 100, Cost: 2.372007
Epoch 2 / 100, Cost: 0.725470
Epoch 3 / 100, Cost: 0.675219
Epoch 4 / 100, Cost: 0.677012
Epoch 5 / 100, Cost: 0.645114
Epoch 6 / 100, Cost: 0.635742
Epoch 7 / 100, Cost: 0.622053
Epoch 8 / 100, Cost: 0.615611
Epoch 9 / 100, Cost: 0.557917
Epoch 10 / 100, Cost: 0.605357
Epoch 11 / 100, Cost: 0.529028
Epoch 12 / 100, Cost: 0.557794
Epoch 13 / 100, Cost: 0.470702
Epoch 14 / 100, Cost: 0.524688
Epoch 15 / 100, Cost: 0.565508
Epoch 16 / 100, Cost: 0.477859
Epoch 17 / 100, Cost: 0.399513
Epoch 18 / 100, Cost: 0.454787
Epoch 19 / 100, Cost: 0.426855
Epoch 20 / 100, Cost: 0.361430
Epoch 21 / 100, Cost: 0.316375
Epoch 22 / 100, Cost: 0.412758
Epoch 23 / 100, Cost: 0.357465
Epoch 24 / 100, Cost: 0.370292
Epoch 25 / 100, Cost: 0.263527
Epoch 26 / 100, Cost: 0.257853
Epoch 27 / 100, Cost: 0.182289
Epoch 28 / 100, Cost: 0.216509
Epoch 29 / 100, Cost: 0.237303
Epoch 30 / 100, Cost: 0.233547
Epoch 31 / 100, Cost: 0.300460
Epoch 32 / 100, Cost: 0.167145
Epoch 33 / 100, C

## Weight Save

In [83]:
torch.save(net.state_dict(), "./models/catsanddogs_cnn.pth")

In [84]:
new_net = CNN().to(device)

## Weight load

In [85]:
new_net.load_state_dict(torch.load('./models/catsanddogs_cnn.pth'))

<All keys matched successfully>

In [86]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])
net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([ 0.3255,  0.2246,  0.1672,  0.0831, -0.3185], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([ 0.3255,  0.2246,  0.1672,  0.0831, -0.3185], device='cuda:0',
       grad_fn=<SelectBackward>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]], device='cuda:0')

## Define torchvision.transforms property (To.Tensor and Resize) to Test

In [87]:
trans = torchvision.transforms.Compose([
    transforms.Resize((64, 128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root = './catsanddogs/test_data', transform = trans)

## Define dataloader

In [88]:
test_set = torch.utils.data.DataLoader(dataset = test_data, batch_size = len(test_data))

## Check accuracy

In [89]:
with torch.no_grad():
    for X, Y in test_set:

        X = X.to(device)
        Y = Y.to(device)

        prediction = new_net(X)

        correct_prediction = torch.argmax(prediction, 1) == Y
        accuracy = correct_prediction.float().mean()

    print('Accuracy: {:.6f}'.format(accuracy.item()))

Accuracy: 0.650000
